## Imports

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

## Part I: Exercises

### Exercise 1

### Exercise 2

## Part II: Practical Applications

### K-means Clustering

### Hierarchical Clustering